In [1]:
# Includes, global variables, initial preprocessing (Dataset folder parsing)

from tensorflow.keras.layers import Dense, Input, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D, CuDNNLSTM, ZeroPadding3D, TimeDistributed
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
#from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.optimizers import Nadam

import matplotlib.pyplot as plot
import tensorflow as tf
import numpy as np
import pickle
import random
import ntpath
import time
import cv2
import os

# Folder to the UFC Dataset
UFC_FOLDER = "/home/box/Downloads/UCF-Anomaly-Detection-Dataset/"
# Last # videos from each folder to be used for testing
NUM_FILES_FOR_TESTING = 5
# Categories for videos
CATEGORIES = ["Normal", "Anomaly"]
# Resolution to use (ex: 100x100)
FRAME_REZ = 100
# Number of frames in one batch
NUM_FRAMES_IN_BATCH = 100
# Skip factor (ex: if set to 4, then remove first quater of frames from batch,
#   shift everything to the begining and populate created space with new frames)
SKIP_FACTOR = 1.25 # 4

def getListOfAnomFolders():
    folders = []
    for folder in os.listdir(UFC_FOLDER):
        if "Normal" not in folder:
            folders.append(folder)
    return folders

def getTrainingVids(anomFolders):
    train_vids = []

    # Add all tuples of (normal categorie, normal videos paths) to the training list
    normal_vids = os.path.join(UFC_FOLDER, "Normal_Videos_event")
    for vid in os.listdir(normal_vids):
        if "mp4" in vid:
            train_vids.append((CATEGORIES.index("Normal"), os.path.join(normal_vids, vid)))
            pass

    # Add all tuples of (normal categorie, normal videos paths) to the training list
    normal_vids = os.path.join(UFC_FOLDER, "Training_Normal_Videos_Anomaly")
    for vid in os.listdir(normal_vids):
        if "mp4" in vid:
            train_vids.append((CATEGORIES.index("Normal"), os.path.join(normal_vids, vid)))

    # Add all tuples of (anomalous categorie, anomalous video path) to the reaining list
    for anomFolder in anomFolders:
        tmp_fol = os.path.join(UFC_FOLDER, anomFolder)
        for vid in os.listdir(tmp_fol)[:-NUM_FILES_FOR_TESTING]:
            if "mp4" in vid:
                train_vids.append((CATEGORIES.index("Anomaly"), os.path.join(tmp_fol, vid)))
                
    # Randomise list for better training
    random.shuffle(train_vids)
    return train_vids

def getTestingVids(anomFolders):
    test_vids = []

    # Add all tuples of (normal categorie, normal videos paths) to the testing list
    normal_vids = os.path.join(UFC_FOLDER, "Testing_Normal_Videos_Anomaly")
    for vid in os.listdir(normal_vids):
        if "mp4" in vid:
            test_vids.append((CATEGORIES.index("Normal"), os.path.join(normal_vids, vid)))

    # Add all tuples of (anomalous categorie, anomalous video path) to the reaining list
    for anomFolder in anomFolders:
        tmp_fol = os.path.join(UFC_FOLDER, anomFolder)
        for vid in os.listdir(tmp_fol)[-NUM_FILES_FOR_TESTING:]:
            if "mp4" in vid:
                test_vids.append((CATEGORIES.index("Anomaly"), os.path.join(tmp_fol, vid)))

    # Randomise list for better training
    random.shuffle(test_vids)
    return test_vids

UFC_FOLDER = os.path.join(UFC_FOLDER, "UCF_Crimes/Videos")
anomFolders = getListOfAnomFolders()
training_vid_paths = getTrainingVids(anomFolders)
testing_vid_paths = getTestingVids(anomFolders)
print("Number of videos for training: ", len(training_vid_paths))
print("Number of videos for testing: ", len(testing_vid_paths))

Number of videos for training:  1735
Number of videos for testing:  215


In [2]:
# Model No.1 input 25x100x100x

model = Sequential()

video = Input(shape=(NUM_FRAMES_IN_BATCH//4,
                     FRAME_REZ, #in the example: channels,
                     FRAME_REZ, #rows,
                     3))        #in the example: columns))
cnn_base = VGG16(input_shape=(FRAME_REZ, #channels,
                              FRAME_REZ, #rows,
                              3),        #columns),
                 weights="imagenet",
                 include_top=False)
cnn_out = GlobalAveragePooling2D()(cnn_base.output)
cnn = Model(cnn_base.input, cnn_out)
cnn.trainable = False
encoded_frames = TimeDistributed(cnn)(video)
encoded_sequence = CuDNNLSTM(256)(encoded_frames)
hidden_layer = Dense(1024, activation="relu")(encoded_sequence)
outputs = Dense(1, activation="sigmoid")(hidden_layer)
model = Model([video], outputs)

optimizer = Nadam(lr=0.002,
                  beta_1=0.9,
                  beta_2=0.999,
                  epsilon=1e-08,
                  schedule_decay=0.004)
model.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [3]:
# Training on batches of size 100x100x100x3 by extracting them from video in real-time

NAME = "new-1epochs-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

# Training function
def train(data_X, data_y):
    # Shuffle data_X and data_y, but make sure each element corresponds to the right label
    # Source: https://stackoverflow.com/questions/23289547/shuffle-two-list-at-once-with-same-order
    c = list(zip(data_X, data_y))
    random.shuffle(c)
    data_X, data_y = zip(*c)
    
    # Shape of each image is ANYxNUM_FRAMES_IN_BATCHxFRAME_REZxFRAME_REZx1, since it's a grayscalse
    # Shape of each image is ANYxNUM_FRAMES_IN_BATCHxFRAME_REZxFRAME_REZx3, since it's not a grayscalse
    data_X = np.array(data_X)
    data_X = data_X.reshape(-1, NUM_FRAMES_IN_BATCH, FRAME_REZ, FRAME_REZ, 3)
    print("data_X shape: ", data_X.shape)
    # Convert data to be between 0 and 1
    #data_X = data_X.astype('float32')
    #data_X /= 255
    
    try:
        # Incrementally train the model
        # This works according to https://github.com/keras-team/keras/issues/4446
        model.fit(data_X, data_y, batch_size=1, epochs=1, validation_split=0.1, callbacks=[tensorboard])
    except Exception as e:
        print("Something went wrong: ", e)
        pass # I know, right?

X = []
y = []
progress = 1

# Will train on 1 video at a time
# TODO: fix this code from breaking during training on larger videos by training on parts of a video at a time rather than on an entire video
for label, vid_path in training_vid_paths:
    print("Working on a {} video ({}/{})".format(CATEGORIES[label], progress, len(training_vid_paths)))
    progress += 1

    # Open a video
    cap = cv2.VideoCapture(vid_path)
    # Get a number of frames in a video
    nframe = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # We will be ignoring leftover frames
    nframe -= nframe % NUM_FRAMES_IN_BATCH
    print("Num frames: ", nframe)
    framearray = []
    batch_limit = 0

    # Only train on bigger video files
    if nframe <= NUM_FRAMES_IN_BATCH*4:
        continue
    
    # Only use every 4th frame
    for i in range((nframe-NUM_FRAMES_IN_BATCH)//4):
        # Skip first batch of frames for better samples
        cap.set(cv2.CAP_PROP_POS_FRAMES, i*4+NUM_FRAMES_IN_BATCH)
        ret, frame = cap.read()
        # Resize a frame to proper rezolution
        frame = cv2.resize(frame, (FRAME_REZ, FRAME_REZ))
        # Convert to grayscale
        #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Convert to normal RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        image = img_to_array(frame)
        # reshape to FRAME_REZxFRAME_REZxchannels
        image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
        frame = preprocess_input(image)
        
        # Add to the list
        framearray.append(frame)

        if len(framearray) == NUM_FRAMES_IN_BATCH:
            X.append(framearray)
            y.append(label)
            batch_limit+=1

            # Remove first 3/4ths and shift eveything back
            framearray = framearray[int(NUM_FRAMES_IN_BATCH/SKIP_FACTOR):]

        # To handle large videos
        if batch_limit >= 10:
            # Train on 20 bathes of a video at a time
            train(X, y)
            batch_limit = 0
            X = []
            y = []
            continue

    cap.release()

    train(X, y)

    X = []
    y = []

model.save("{}.model".format(NAME))

Working on a Normal video (1/1735)
Num frames:  8000
data_X shape:  (10, 100, 100, 100, 3)
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 13s 1s/step - loss: 0.1298 - acc: 0.8889 - val_loss: 1.0592e-04 - val_acc: 1.0000
data_X shape:  (10, 100, 100, 100, 3)
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 6s 698ms/step - loss: 2.5551e-05 - acc: 1.0000 - val_loss: 1.7788e-06 - val_acc: 1.0000
data_X shape:  (4, 100, 100, 100, 3)
Train on 3 samples, validate on 1 samples
Epoch 1/1
3/3 [==============================] - 2s 761ms/step - loss: 1.4450e-06 - acc: 1.0000 - val_loss: 6.8226e-07 - val_acc: 1.0000
Working on a Anomaly video (2/1735)
Num frames:  11100
data_X shape:  (10, 100, 100, 100, 3)
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 6s 696ms/step - loss: 2.2276 - acc: 0.6667 - val_loss: 0.0209 - val_acc: 1.0000
data_X shape:  (10, 100, 100, 100, 3)
T

ValueError: not enough values to unpack (expected 2, got 0)

In [66]:
# displaying a frame from a broken up batch to make sure it was saved properly

# Resahpe data back to the image
X = pickle.load(open("/home/box/Documents/video/data/Normal/Normal_Videos001_x264.mp4-0.pickle", "rb"))
X = X * 255
X = X.astype('int32')
X = X.reshape(-1, 100, 100)
for x in X[:1]:
    plot.imshow(x, cmap=plot.cm.binary)
    #img = array_to_img(x, data_format='channels_last')
    #img = preprocess_input(img)
    #print(img.shape)

TypeError: 'Image' object is not subscriptable

In [ ]:
# Break up videos into the fragments of 100x100x100x3 size

progress = 1

# Will train on 1 video at a time
# TODO: fix this code from breaking during training on larger videos by training on parts of a video at a time rather than on an entire video
for label, vid_path in training_vid_paths:
    print("Working on a {} video ({}/{})".format(vid_path, progress, len(training_vid_paths)))
    progress += 1

    # Open a video
    cap = cv2.VideoCapture(vid_path)
    # Get a number of frames in a video
    nframe = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # We will be ignoring leftover frames
    nframe -= nframe % NUM_FRAMES_IN_BATCH
    #print("Num frames: ", nframe)
    framearray = []
    cnt = 0
    
    # Only train on bigger video files
    if nframe <= NUM_FRAMES_IN_BATCH*4:
        continue
    
    # Only use every 4th frame
    for i in range((nframe-NUM_FRAMES_IN_BATCH)//4):
        # Skip first batch of frames for better samples
        cap.set(cv2.CAP_PROP_POS_FRAMES, i*4+NUM_FRAMES_IN_BATCH-1)
        ret, frame = cap.read()
        # Resize a frame to proper rezolution
        frame = cv2.resize(frame, (FRAME_REZ, FRAME_REZ))
        # Convert to grayscale
        #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Convert to normal RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        image = img_to_array(frame)
        # reshape to FRAME_REZxFRAME_REZxchannels
        image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
        #frame = preprocess_input(image)
        
        # Add to the list
        framearray.append(frame)

        if len(framearray) == NUM_FRAMES_IN_BATCH:
            # Save training/testing data
            pickle_out = open("./data/train/{}-{}.pickle".format(ntpath.basename(vid_path), cnt), "wb")
            pickle.dump(framearray, pickle_out)
            pickle_out.close()
            # Remove first 3/4ths and shift eveything back
            framearray = framearray[int(NUM_FRAMES_IN_BATCH/SKIP_FACTOR):]
            cnt += 1
            
    cap.release()

Working on a /home/box/Downloads/UCF-Anomaly-Detection-Dataset/UCF_Crimes/Videos/RoadAccidents/RoadAccidents127_x264.mp4 video (1/1735)
Working on a /home/box/Downloads/UCF-Anomaly-Detection-Dataset/UCF_Crimes/Videos/Robbery/Robbery111_x264.mp4 video (2/1735)
Working on a /home/box/Downloads/UCF-Anomaly-Detection-Dataset/UCF_Crimes/Videos/RoadAccidents/RoadAccidents147_x264.mp4 video (3/1735)
Working on a /home/box/Downloads/UCF-Anomaly-Detection-Dataset/UCF_Crimes/Videos/Training_Normal_Videos_Anomaly/Normal_Videos865_x264.mp4 video (4/1735)
Working on a /home/box/Downloads/UCF-Anomaly-Detection-Dataset/UCF_Crimes/Videos/Shoplifting/Shoplifting050_x264.mp4 video (5/1735)
Working on a /home/box/Downloads/UCF-Anomaly-Detection-Dataset/UCF_Crimes/Videos/Abuse/Abuse028_x264.mp4 video (6/1735)
Working on a /home/box/Downloads/UCF-Anomaly-Detection-Dataset/UCF_Crimes/Videos/Fighting/Fighting026_x264.mp4 video (7/1735)
Working on a /home/box/Downloads/UCF-Anomaly-Detection-Dataset/UCF_Crimes

In [26]:
# train input 25x100x100x3

NAME = "new-vgg-test-1epochs-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

# Training function
def train(data_X, data_y):
    # Shuffle data_X and data_y, but make sure each element corresponds to the right label
    # Source: https://stackoverflow.com/questions/23289547/shuffle-two-list-at-once-with-same-order
    c = list(zip(data_X, data_y))
    random.shuffle(c)
    data_X, data_y = zip(*c)
    
    # Shape of each image is ANYxNUM_FRAMES_IN_BATCHxFRAME_REZxFRAME_REZx1, since it's a grayscalse
    # Shape of each image is ANYxNUM_FRAMES_IN_BATCHxFRAME_REZxFRAME_REZx3, since it's not a grayscalse
    data_X = np.array(data_X)
    data_X = data_X.reshape(-1, NUM_FRAMES_IN_BATCH, FRAME_REZ, FRAME_REZ, 3)
    print("data_X shape: ", data_X.shape)
    # Convert data to be between 0 and 1
    #data_X = data_X.astype('float32')
    #data_X /= 255
    
    try:
        # Incrementally train the model
        # This works according to https://github.com/keras-team/keras/issues/4446
        model.fit(data_X, data_y, batch_size=1, epochs=1, validation_split=0.1, callbacks=[tensorboard])
    except Exception as e:
        print("Something went wrong: ", e)
        pass # I know, right?

class Vid_Generator(Sequence):
    def __init__(self, vid_path, batch_size):
        self.batch_size = batch_size
        self.X = []

        temp = []

        for vid in os.listdir(vid_path):
            temp.append(os.path.join(vid_path, vid))

        random.shuffle(temp)
        self.steps = len(temp) // self.batch_size
        self.X = temp

    def __len__(self):
        return int(np.ceil(len(self.X) / float(self.batch_size))) - 1

    def __getitem__(self, idx):
        batch_x_files = self.X[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = []

        batch_x = []
        for x in batch_x_files:
            # Read batches of frames
            tmp_batch = pickle.load(open(x, "rb"))
            fin_batch = []
            cnt = 0
            # For each frame in a batch
            for tmp_frame in tmp_batch:
                # Convert it to float
                tmp_frame = preprocess_input(tmp_frame)
                # Add processed frame to new batch
                fin_batch.append(tmp_frame)
                cnt += 1
                
                if cnt == 25:
                    if "Normal" in x:
                        batch_y.append(CATEGORIES.index("Normal"))
                    else:
                        batch_y.append(CATEGORIES.index("Anomaly"))
                        
                    # Collect all batches together
                    batch_x.append(fin_batch)
                    fin_batch = []
                    cnt = 0
            
        batch_x = np.array(batch_x)
        #print("Loaded shape: ", batch_x.shape)
        batch_x = batch_x.reshape(-1, batch_x.shape[1], batch_x.shape[2], batch_x.shape[3], batch_x.shape[4])
        #print("Final shape: ", batch_x.shape)

        return np.array(batch_x), np.array(batch_y)

BATCH_SIZE = 1
training_generator = Vid_Generator("./data/train/", BATCH_SIZE)
testing_generator = Vid_Generator("./data/test/", BATCH_SIZE)

model.fit_generator(generator=training_generator, 
                    steps_per_epoch=training_generator.steps, 
                    epochs=1, verbose=1, 
                    validation_data=testing_generator, 
                    validation_steps=testing_generator.steps, 
                    callbacks=[tensorboard])

model.save("{}.model".format(NAME))

Epoch 1/1
Loaded shape:  (1, 100, 100, 100, 3)


ValueError: Error when checking target: expected dense_5 to have shape (2,) but got array with shape (1,)

Loaded shape:  (1, 100, 100, 100, 3)


In [8]:
# test input 25x100x100x3

test_model = tf.keras.models.load_model("new-vgg-test-1epochs-1553182342.model")
temp = []
for vid in os.listdir("./data/test/"):
    temp.append(os.path.join("./data/test/", vid))

random.shuffle(temp)
for x in temp:
    # Read batches of frames
    tmp_batch = pickle.load(open(x, "rb"))
    fin_batch = []
    cnt = 0
    
    # For each frame in a batch
    for tmp_frame in tmp_batch:
        # Convert it to float
        tmp_frame = preprocess_input(tmp_frame)
        # Add processed frame to new batch
        fin_batch.append(tmp_frame)
        cnt += 1
        
        if cnt == 25:
            y = []
            if "Normal" in x:
                ind = CATEGORIES.index("Normal")
                y = [1, 0]
            else:
                ind = CATEGORIES.index("Anomaly")
                y = [0, 1]
            
            fin_batch = np.array(fin_batch)
            fin_batch = fin_batch.reshape(1, fin_batch.shape[0], fin_batch.shape[1], fin_batch.shape[2], fin_batch.shape[3])

            predict = test_model.predict(fin_batch)
            print("Looking at a {} video, prediction: {}".format(CATEGORIES[ind], predict))
            
            # Collect all batches together
            fin_batch = []
            cnt = 0

Looking at a Anomaly video, prediction: [[0.27860102]]
Looking at a Anomaly video, prediction: [[0.27860102]]
Looking at a Anomaly video, prediction: [[0.27860102]]
Looking at a Anomaly video, prediction: [[0.27860102]]


In [4]:
# Model No.2 input 100x100x100x3

video = Input(shape=(NUM_FRAMES_IN_BATCH,
                     FRAME_REZ, #in the example: channels,
                     FRAME_REZ, #rows,
                     3))        #in the example: columns))
cnn_base = VGG16(input_shape=(FRAME_REZ, #channels,
                              FRAME_REZ, #rows,
                              3),        #columns),
                 weights="imagenet",
                 pooling="avg",
                 include_top=False)

cnn_out = cnn_base.output
cnn = Model(cnn_base.input, cnn_out)
cnn.trainable = False

encoded_frames = TimeDistributed(cnn)(video)
encoded_sequence = CuDNNLSTM(256)(encoded_frames)

hidden_layer = CuDNNLSTM(256)(Dropout(0.25)(Dense(1024, activation="relu")(encoded_sequence)))
hidden_layer = Dropout(0.25)(Dense(1024, activation="relu")(hidden_layer))
outputs = Dense(1, activation="sigmoid")(hidden_layer)
model = Model([video], outputs)

optimizer = Nadam(lr=0.002,
                  beta_1=0.9,
                  beta_2=0.999,
                  epsilon=1e-08,
                  schedule_decay=0.004)
model.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

ValueError: Input 0 of layer cu_dnnlstm_2 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 1024]

In [ ]:
# train input 100x100x100x3

NAME = "100frames-2lstm-vgg-test-10epochs-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

# Training function
def train(data_X, data_y):
    # Shuffle data_X and data_y, but make sure each element corresponds to the right label
    # Source: https://stackoverflow.com/questions/23289547/shuffle-two-list-at-once-with-same-order
    c = list(zip(data_X, data_y))
    random.shuffle(c)
    data_X, data_y = zip(*c)
    
    # Shape of each image is ANYxNUM_FRAMES_IN_BATCHxFRAME_REZxFRAME_REZx1, since it's a grayscalse
    # Shape of each image is ANYxNUM_FRAMES_IN_BATCHxFRAME_REZxFRAME_REZx3, since it's not a grayscalse
    data_X = np.array(data_X)
    data_X = data_X.reshape(-1, NUM_FRAMES_IN_BATCH, FRAME_REZ, FRAME_REZ, 3)
    print("data_X shape: ", data_X.shape)
    # Convert data to be between 0 and 1
    #data_X = data_X.astype('float32')
    #data_X /= 255
    
    try:
        # Incrementally train the model
        # This works according to https://github.com/keras-team/keras/issues/4446
        model.fit(data_X, data_y, batch_size=1, epochs=1, validation_split=0.1, callbacks=[tensorboard])
    except Exception as e:
        print("Something went wrong: ", e)
        pass # I know, right?

class Vid_Generator(Sequence):
    def __init__(self, vid_path, batch_size):
        self.batch_size = batch_size
        self.X = []

        temp = []

        for vid in os.listdir(vid_path):
            temp.append(os.path.join(vid_path, vid))

        random.shuffle(temp)
        self.X = temp
        self.steps = len(self.X)
        print(self.steps)

    def __len__(self):
        return int(np.ceil(len(self.X) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x_files = self.X[idx:idx + 1]

        batch_x = []
        batch_y = []
        
        for x in batch_x_files:
            if "Normal" in x:
                batch_y.append(CATEGORIES.index("Normal"))
            else:
                batch_y.append(CATEGORIES.index("Anomaly"))
                        
            # Read batches of frames
            tmp_batch = pickle.load(open(x, "rb"))
            fin_batch = []

            # For each frame in a batch
            for tmp_frame in tmp_batch:
                # Convert it to float
                tmp_frame = preprocess_input(tmp_frame)
                # Add processed frame to new batch
                fin_batch.append(tmp_frame)
                
            # Collect all batches together
            batch_x.append(fin_batch)
            
        batch_x = np.array(batch_x)
        #print("Loaded shape: ", batch_x.shape)
        #batch_x = batch_x.reshape(-1, batch_x.shape[1], batch_x.shape[2], batch_x.shape[3], batch_x.shape[4])
        #print("Final shape: ", batch_x.shape)

        return np.array(batch_x), np.array(batch_y)

BATCH_SIZE = 1
training_generator = Vid_Generator("./data/train/", BATCH_SIZE)
testing_generator = Vid_Generator("./data/test/", BATCH_SIZE)

model.fit_generator(generator=training_generator, 
                    steps_per_epoch=training_generator.steps, 
                    epochs=10, verbose=1, 
                    validation_data=testing_generator, 
                    validation_steps=testing_generator.steps, 
                    callbacks=[tensorboard])

model.save("{}.model".format(NAME))

24094
2545
Epoch 1/10
24094/24094 [==============================] - 16617s 690ms/step - loss: 0.6722 - acc: 0.6029 - val_loss: 0.6233 - val_acc: 0.7257
Epoch 2/10
23543/24094 [============================>.] - ETA: 6:06 - loss: 0.6721 - acc: 0.6031